# Build Pipeline with Components from yaml

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace. [Check this notebook for creating a workspace](/sdk/resources/workspace/workspace.ipynb) 
- A Compute Cluster. [Check this notebook to create a compute cluster](/sdk/resources/compute/compute.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](/sdk/README.md#getting-started)

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Define and load `CommandComponent` from YAML
- Create `Pipeline` using loaded component.

**Motivations** - This notebook covers the scenario that user define components using yaml then use these components to build pipeline.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [1]:
# Import required libraries
from azure.ml import MLClient, dsl
from azure.ml.entities import JobInput, load_component
from azure.ml._constants import AssetTypes
import piprunpkg

## 1.2. Configure credential

We are using `DefaultAzureCredential` to get access to workspace. When an access token is needed, it requests one using multiple identities(`EnvironmentCredential, ManagedIdentityCredential, SharedTokenCacheCredential, VisualStudioCodeCredential, AzureCliCredential, AzurePowerShellCredential`) in turn, stopping when one provides a token.
Reference [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for more information.

`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 
Reference [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python) for all available credentials if it does not work for you.  

In [ ]:
from azure.identity import DefaultAzureCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token('https://management.azure.com/.default')
except Exception as ex:
    # If exception happens when retrieve token, try exclude the failed credential like this then try again:
    # Exclude VSCode credential:
    # credential = DefaultAzureCredential(exclude_visual_studio_code_credential=True)
    raise Exception("Failed to retrieve a token from the included credentials due to the following exception, try to add `exclude_xxx_credential=True` to `DefaultAzureCredential` and try again.") from ex

## 1.3. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ml` to get a handle to the required Azure Machine Learning workspace.

In [ ]:
try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    # NOTE: Update following workspace information if not correctly configure before
    client_config = {
        "subscription_id": "<SUBSCRIPTION_ID>",
        "resource_group": "<RESOURCE_GROUP>",
        "workspace_name": "<WORKSPACE_NAME>"
    }

    if client_config["subscription_id"].startswith('<'):
        print("please update your <SUBSCRIPTION_ID> <RESOURCE_GROUP> <WORKSPACE_NAME> in notebook cell")
        raise ex
    else:  # write and reload from config file
        import json, os
        config_path = "../../.azureml/config.json"
        os.makedirs(os.path.dirname(config_path), exist_ok=True)
        with open(config_path, "w") as fo:
            fo.write(json.dumps(client_config))
        ml_client = MLClient.from_config(credential=credential, path=config_path)
print(ml_client)

## 1.4. Retrieve or create an Azure Machine Learning compute target

In [ ]:
# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
try:
    ml_client.compute.get(name=cluster_name)
except Exception:
    print('Creating a new compute target...')
    from azure.ml.entities import AmlCompute
    compute = AmlCompute(
        name=cluster_name,
        size="Standard_D2_v2",
        max_instances=2
    )
    ml_client.compute.begin_create_or_update(compute)

# 2. Define and create components into workspace
## 2.1 Load components from YAML

In [2]:
parent_dir = '.'
#parent_dir = "D:\\practice\\azureml-examples\\sdk\\jobs\\pipelines\\1a_pipeline_with_components_from_yaml"
train_model = load_component(yaml_file=parent_dir + "/train_model.yml")
score_data = load_component(yaml_file=parent_dir + "/score_data.yml")
eval_model = load_component(yaml_file=parent_dir + "/eval_model.yml")

## 2.2 Inspect loaded component

In [3]:
# Print the component as yaml
print(train_model)

$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: train_model
version: 0.0.1
display_name: Train Model
description: A dummy training component
type: command
inputs:
  training_data:
    type: uri_folder
  max_epochs:
    type: integer
    optional: true
  learning_rate:
    type: number
    default: '0.01'
  learning_rate_schedule:
    type: string
    default: time-based
outputs:
  model_output:
    type: uri_folder
command: python train.py  --training_data ${{inputs.training_data}}  [--max_epochs
  ${{inputs.max_epochs}}]  --learning_rate ${{inputs.learning_rate}}  --learning_rate_schedule
  ${{inputs.learning_rate_schedule}}  --model_output ${{outputs.model_output}}
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:5
code: azureml:./train_src
is_deterministic: true
tags: {}



In [4]:
# Inspect more information
print(type(train_model))
help(train_model._func)

<class 'azure.ml.entities._component.command_component.CommandComponent'>
Help on function [component] Train Model:

[component] Train Model(*, training_data: 'uri_folder' = None, max_epochs: 'int' = None, learning_rate: 'float' = None, learning_rate_schedule: 'str' = None)
    A dummy training component
    
    Component yaml:
    ```yaml
    $schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
    type: command
    
    name: train_model
    display_name: Train Model
    description: A dummy training component
    version: 0.0.1
    inputs:
      training_data: 
        type: uri_folder
      max_epochs:
        type: integer
        optional: true
      learning_rate: 
        type: number
        default: 0.01
      learning_rate_schedule: 
        type: string
        default: time-based 
    outputs:
      model_output:
        type: uri_folder
    code: ./train_src
    environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:5
    command: >-


# 3. Sample pipeline job
## 3.1 Build pipeline

In [5]:
# Construct pipeline
@dsl.pipeline(
    default_compute="cpu-cluster",
    description="E2E dummy train-score-eval pipeline with components defined via yaml",
)
def pipeline_with_components_from_yaml(
    training_input,
    test_input,
    training_max_epochs=20,
    training_learning_rate=1.8,
    learning_rate_schedule="time-based",
):
    # Call component obj as function: apply given inputs & parameters to create a node in pipeline
    train_with_sample_data = train_model(
        training_data=training_input,
        max_epochs=training_max_epochs,
        learning_rate=training_learning_rate,
        learning_rate_schedule=learning_rate_schedule,
    )

    score_with_sample_data = score_data(
        model_input=train_with_sample_data.outputs.model_output, 
        test_data=test_input
    )
    score_with_sample_data.outputs.score_output.mode = "upload"

    eval_with_sample_data = eval_model(
        scoring_result=score_with_sample_data.outputs.score_output
    )

    # Return: pipeline outputs
    return {
        "trained_model": train_with_sample_data.outputs.model_output,
        "scored_data": score_with_sample_data.outputs.score_output,
        "evaluation_report": eval_with_sample_data.outputs.eval_output,
    }

pipeline = pipeline_with_components_from_yaml(
    training_input = JobInput(type=AssetTypes.URI_FOLDER, path=parent_dir + "/data/"),
    test_input = JobInput(type=AssetTypes.URI_FOLDER, path=parent_dir + "/data/"),
    training_max_epochs=20,
    training_learning_rate=1.8,
    learning_rate_schedule="time-based"
)

In [6]:
# Inspect built pipeline
print(pipeline)
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
from piprunpkg import run_pipeline, CuratedEnvironmentDefaultRunMode
print(os.getcwd())
output_root_dir = "./local-run-output/notebook_1a_native"
run_pipeline(output_root_dir=output_root_dir, pipeline_job=pipeline, curated_env_default_run_mode=CuratedEnvironmentDefaultRunMode.CONTAINER)

name: helpful_sugar_g160163tmn
display_name: pipeline_with_components_from_yaml
description: E2E dummy train-score-eval pipeline with components defined via yaml
type: pipeline
inputs:
  training_input:
    mode: ro_mount
    type: uri_folder
    path: azureml:./data/
  test_input:
    mode: ro_mount
    type: uri_folder
    path: azureml:./data/
  training_max_epochs: 20
  training_learning_rate: 1
  learning_rate_schedule: time-based
outputs:
  trained_model: null
  scored_data: null
  evaluation_report: null
tags: {}
jobs:
  train_with_sample_data:
    $schema: '{}'
    type: command
    inputs:
      training_data: ${{parent.inputs.training_input}}
      max_epochs: ${{parent.inputs.training_max_epochs}}
      learning_rate: ${{parent.inputs.training_learning_rate}}
      learning_rate_schedule: ${{parent.inputs.learning_rate_schedule}}
    outputs:
      model_output: ${{parent.outputs.trained_model}}
    command: python train.py  --training_data ${{inputs.training_data}}  [--max_

## 3.2 Submit pipeline job

In [ ]:
# Submit pipeline job to workspace
pipeline_job = ml_client.jobs.create_or_update(pipeline, experiment_name="pipeline_samples")
pipeline_job

In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

# Next Steps
You can see further examples of running a pipeline job [here](/sdk/jobs/pipelines/README.md)